<div style="background-color:yellow; text-align:center; text-align:center; padding:40px;">
<h1  style="color:red;" > MMI-714 : Generative Models for Multimedia </h1>   
<h2  style="color:red;" > Final Report </h2>
<br>
<h3  style="color:red;  font-style:italic;" > Exploration of the Intuitive Physics through the 
Latent Space Disentanglement</h3>
<br>
<h4  style="color:red;" >Turgay Yıldız</h4>
<br>
<h4  style="color:red;" >Cognitive Sciences,  Middle East Technical University (METU)</h4>
</div>


<div span style="background-color:yellow;    color:red;      text-align:center;    padding:5px;">
<h2>  First Import the Relevant Packages  </h2>
</div>

In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset, random_split
import torchvision.transforms as transforms
import numpy as np
import torch.optim as optim
import matplotlib.pyplot as plt
from torch.utils.checkpoint import checkpoint 
from torch.amp import GradScaler, autocast
import torch.optim.lr_scheduler as lr_scheduler
import torch.nn.functional as F
from torch.nn.functional import relu
import torch.nn.init as init
from torchvision import models
from torch.amp import GradScaler, autocast
from PIL import Image
from einops.layers.torch import Rearrange
from einops import repeat
from torch import Tensor
import itertools
import tqdm
import pandas as pd
import cv2
import os 

<div span style="background-color:yellow;    color:red;      text-align:center;    padding:5px;">
<h2>  Load the Dataset  </h2>
</div>

In [ ]:
class ImageDataset(Dataset):
    def __init__(self, img_dir):
        
        self.img_dir     =    img_dir
        
        self.img_files   =    sorted(os.listdir(img_dir))
        
        
    def __len__(self):
        return len(self.img_files)

    def __getitem__(self, idx):
        
        img_path = os.path.join(self.img_dir, self.img_files[idx])

        img         = Image.open(img_path).convert("RGB")  # Ensure 3-channel RGB
        img_array   = np.array(img, dtype=np.float32) / 255.0  # Normalize to [0, 1]
        img_tensor  = torch.tensor(img_array).reshape(3, 224, 224)  # Convert to PyTorch tensor and reshape to (C, H, W)


        return img_tensor

In [ ]:
img_path = "/home/turgay/falling_objects_dataset/img_files/"

In [ ]:
sorted(os.listdir(img_path))[:3]

In [ ]:
dataset = ImageDataset(img_dir=img_path)

In [ ]:
len(dataset)

In [ ]:
train_size =  int(0.8 * len(dataset))
val_size   =  len(dataset) - train_size

In [ ]:
train_dataset, val_dataset = random_split(dataset, [train_size, val_size]) 

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader   = DataLoader(val_dataset,   batch_size=64, shuffle=True)

In [ ]:
for i in train_loader:
    print(i.shape)
    break

In [ ]:
i.max(), i.min()

In [ ]:
i  =  i.reshape(-1, 224, 224, 3)

In [ ]:
path = "/home/turgay/Turgay/Academic/2024-2025/Fall/Generative_Models/Final_Project/Data/6_pairs/" 

In [ ]:
path_data = path + "concatenated_data.npy"

In [ ]:
X = np.load(path_data, mmap_mode="r")

In [ ]:
X.shape

In [ ]:
X  =  X.reshape(3529, 6, 3, 224, 224)

In [ ]:
X_image =  X[:, 0, :, :, :] 
X_color =  X[:, 1, :, :, :] 
X_order =  X[:, 2, :, :, :] 

X_image2 =  X[:, 3, :, :, :]
X_color2 =  X[:, 4, :, :, :]
X_order2 =  X[:, 5, :, :, :]

In [ ]:
X_image.shape

In [ ]:
class ImageDataset(Dataset):
    def __init__(self, X):
        
        self.X     =    X
 
    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        
        img_array   = np.array(self.X[idx], dtype=np.float32) / 255.0         # Normalize to [0, 1]
        img_tensor  = torch.tensor(img_array)                            # Convert to PyTorch tensor and reshape to (C, H, W)


        return img_tensor

In [ ]:
dataset = ImageDataset(X_image)

In [ ]:
len(dataset)

In [ ]:
train_size =  int(0.8 * len(dataset))
val_size   =  len(dataset) - train_size

In [ ]:
train_dataset, val_dataset = random_split(dataset, [train_size, val_size]) 

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader   = DataLoader(val_dataset,   batch_size=64, shuffle=True)

In [ ]:
for i in train_loader:
    print(i.shape)
    break

In [ ]:
i  =  i.reshape(-1, 224, 224, 3)

<div span style="background-color:yellow;    color:red;      text-align:center;    padding:5px;">
<h2>  Function to Plot Images :  </h2>
</div>

In [ ]:
def plot_img(data, row, col, size1, size2, c_map = None): 
    
    if (row == 1) and (col == 1): 
        fig, ax = plt.subplots(1, 1, figsize=(size1, size2))
        ax.imshow(data, cmap=c_map) 
        ax.set_axis_off() 

    elif (row == 1) and (col > 1):
        fig, axes = plt.subplots(1, col, figsize=(size1, size2))
        for i in range(col):
            axes[i].imshow(data[i], cmap=c_map) 
            axes[i].set_axis_off() 
            axes[i].set_title(f"Image {i}")

    else:
        fig, axes = plt.subplots(row, col, figsize=(size1, size2))
        axes = axes.flatten() 
        for i in range(row * col):
            if i < len(data):  
                axes[i].imshow(data[i], cmap=c_map) 
                axes[i].set_axis_off()  
                axes[i].set_title(f"Image {i}")
            else:
                axes[i].axis('off') 

    plt.tight_layout()  
    plt.show()


In [ ]:
plot_img(i[:10], 2, 5, 5, 5) 

<div span style="background-color:yellow;    color:red;      text-align:center;    padding:5px;">
<h2>  Patch Embeddings for the ViT part of the architecture :  </h2>
</div>

In [ ]:
class PatchEmbedding(nn.Module):
    def __init__(self, in_channels = 3, patch_size = 16, embed_dim = 64):
        self.patch_size = patch_size
        super().__init__()
        
        self.projection = nn.Sequential(
            # break-down the image in s1 x s2 patches and flat them
            Rearrange('b c (h p1) (w p2) -> b (h w) (p1 p2 c)', p1=patch_size, p2=patch_size),
            nn.Linear(patch_size * patch_size * in_channels, embed_dim)
        )

    def forward(self, x: Tensor) -> Tensor:
        x = self.projection(x)
        return x

<div span style="background-color:yellow;    color:red;      text-align:center;    padding:5px;">
<h2>  Multi-Head Attention  </h2>
</div>

In [ ]:
class Attention(nn.Module):
    def __init__(self, embed_dim, n_heads, dropout):
        super().__init__()
        
        self.n_heads   =    n_heads
        self.att       =    torch.nn.MultiheadAttention(embed_dim =  embed_dim,
                                                       num_heads  =  n_heads,
                                                       dropout    =  dropout ) 
        self.q = torch.nn.Linear(embed_dim, embed_dim)
        self.k = torch.nn.Linear(embed_dim, embed_dim)
        self.v = torch.nn.Linear(embed_dim, embed_dim) 

    def forward(self, x):
        
        q = self.q(x)
        k = self.k(x)
        v = self.v(x)
        
        attn_output, attn_output_weights = self.att(x, x, x)
        
        return attn_output

In [ ]:
#Attention(embed_dim=256, n_heads=4, dropout=0.)(torch.ones((1, 196, 256))).shape

<div span style="background-color:yellow;    color:red;      text-align:center;    padding:5px;">
<h2>  Normalizing Module : </h2>
</div>

In [ ]:
class PreNorm(nn.Module):
    def __init__(self, embed_dim, fn):
        super().__init__()
        
        self.norm   =   nn.LayerNorm(embed_dim)
        self.fn     =   fn
        
    def forward(self, x, **kwargs):
        
        return self.fn(self.norm(x), **kwargs) 
    

In [ ]:
#norm = PreNorm(embed_dim=256, fn=Attention(embed_dim=256, n_heads=4, dropout=0.))

In [ ]:
#norm(torch.ones((1, 196, 256))).shape

<div span style="background-color:yellow;    color:red;      text-align:center;    padding:5px;">
<h2>  Feed-Forward  </h2>
</div>

In [ ]:
class FeedForward(nn.Sequential):
    
    def __init__(self, embed_dim, hidden_dim, dropout = 0.):    
        
        super().__init__(
            
            nn.Linear(embed_dim, hidden_dim),       #   hidden_dim   =   2   *   embed_dim
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, embed_dim),       
            nn.Dropout(dropout) 
        )

In [ ]:
#ff = FeedForward(embed_dim=256, hidden_dim=512)

In [ ]:
#ff(torch.ones((1, 196, 256))).shape

<div span style="background-color:yellow;    color:red;      text-align:center;    padding:5px;">
<h2>  Residual Connections </h2>
</div>

In [ ]:
class ResidualAdd(nn.Module):
    def __init__(self, fn):
        super().__init__()
        
        self.fn = fn

    def forward(self, x, **kwargs):
        
        res   =   x
        x     =   self.fn(x, **kwargs)
        x    +=   res
        
        return x

In [ ]:
#residual_att = ResidualAdd(Attention(embed_dim=256, n_heads=4, dropout=0.))

In [ ]:
#residual_att(torch.ones((1, 196, 256))).shape

<div span style="background-color:yellow;    color:red;      text-align:center;    padding:5px;">
<h2>  Model </h2>
</div>

In [ ]:
class Beta_VAE_ViT(nn.Module):
    
    def __init__(self, ch=3, img_size=224, patch_size=16, embed_dim=64, latent_dim = 64, n_layers=6, dropout=0.1, heads=8):
        super(Beta_VAE_ViT, self).__init__()

        # Attributes
        self.channels    =   ch
        self.height      =   img_size
        self.width       =   img_size
        self.patch_size  =   patch_size
        self.n_layers    =   n_layers
        self.embed_dim   =   embed_dim
        self.latent_dim  =   latent_dim
        
        
        num_patches      =   (img_size // patch_size) ** 2
        self.num_patches =   num_patches
        
#-----------------------------------------------------------------------------------------------------------
        # Patching
        self.patch_embedding  = PatchEmbedding(in_channels =  ch,
                                               patch_size  =  patch_size,
                                               embed_dim   =  self.embed_dim)
#-----------------------------------------------------------------------------------------------------------        
        self.pos_embedding    =    nn.Parameter(torch.randn(1, num_patches + 1, self.embed_dim))
#-----------------------------------------------------------------------------------------------------------
        # Transformer Encoder
        self.encoder_layers   =    nn.ModuleList([]) 
        
        for _ in range(n_layers):
            transformer_encoder_block = nn.Sequential(
                                            ResidualAdd(PreNorm(self.embed_dim, Attention(self.embed_dim, n_heads = heads, dropout = dropout))),
                                            ResidualAdd(PreNorm(self.embed_dim, FeedForward(self.embed_dim, 2 * self.embed_dim, dropout = dropout)))
                                            )
            self.encoder_layers.append(transformer_encoder_block) #  (self.num_patches,   self.embed_dim)  

#-----------------------------------------------------------------------------------------------------------
        self.fc_encoder_mu = nn.Sequential(
                                        
                                        nn.Linear(self.num_patches * self.embed_dim,   512),
                                        nn.LeakyReLU(),
                                        nn.BatchNorm1d(512),
                                         
                                        nn.Linear(512, 256),
                                        nn.LeakyReLU(),
                                        nn.BatchNorm1d(256),
            
                                        nn.Linear(256,  self.latent_dim),
                                
                                        )
    
        self.fc_encoder_logvar = nn.Sequential(
            
                                        nn.Linear(self.num_patches * self.embed_dim,   512),
                                        nn.LeakyReLU(),
                                        nn.BatchNorm1d(512),
                                         
                                        nn.Linear(512, 256),
                                        nn.LeakyReLU(),
                                        nn.BatchNorm1d(256),
            
                                        nn.Linear(256,  self.latent_dim),
                                     
                                        )
#-----------------------------------------------------------------------------------------------------------
        self.fc_decoder = nn.Sequential(
            
                                        nn.BatchNorm1d(self.latent_dim),
            
                                        nn.Linear(self.latent_dim,         num_patches * 4),
                                        nn.LeakyReLU(),
                                        nn.BatchNorm1d(num_patches * 4),
            
                                        nn.Linear(num_patches * 4,         num_patches * 16),
                                        nn.LeakyReLU(),
                                        nn.BatchNorm1d(num_patches * 16),
            
                                        nn.Linear(num_patches * 16,        num_patches * 64),   #  196 * 64
                                        nn.LeakyReLU(),
                                        nn.BatchNorm1d(num_patches * 64),
                                        )
#-----------------------------------------------------------------------------------------------------------
        self.reconstruct = nn.Sequential(
         
                nn.ConvTranspose2d(self.num_patches, 128, kernel_size=4, stride=2, padding=2),         #   8x8   ->  14x14 
                nn.LeakyReLU(),
                nn.BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
            
                nn.ConvTranspose2d(128,          64, kernel_size=4, stride=2, padding=1),              #   14    ->  28
                nn.LeakyReLU(),
                nn.BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
            
                nn.ConvTranspose2d(64,           32, kernel_size=4, stride=2, padding=1),              #   28    ->  56
                nn.LeakyReLU(),
                nn.BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
            
                nn.ConvTranspose2d(32,           16, kernel_size=4, stride=2, padding=1),              #   56    ->  112
                nn.LeakyReLU(),
                nn.BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
            
                nn.ConvTranspose2d(16,           3,  kernel_size=4, stride=2, padding=1),               #   112   ->  224
                nn.Sigmoid()
        )
#----------------------------------------------------------------------------------------------------------------------------#
#-------------------------------------------           FUNCTIONS               ----------------------------------------------#
#----------------------------------------------------------------------------------------------------------------------------#

    def encode(self, x):
        
        x          =   self.patch_embedding(x)
        b, n, _    =   x.shape 

        x         +=   self.pos_embedding[:, : n]

        # Transformer Encoder layers
        for i in range(self.n_layers):
            x = self.encoder_layers[i](x)
            
        x      =   x.reshape(-1, self.num_patches * self.embed_dim)
            
        mu     = self.fc_encoder_mu(x)
        logvar = self.fc_encoder_logvar(x) 
        
        return mu , logvar
#----------------------------------------------------------------------------------------------------------------------------#
    
    def reparameterize(self, mu, logvar):
        
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std) 
        z   = mu + eps * std 
        
        return z  

#----------------------------------------------------------------------------------------------------------------------------#
      
    def decode(self, z):
        
            x      =   self.fc_decoder(z)
            x      =   x.reshape(-1, self.num_patches, 8, 8)
            x      =   self.reconstruct(x)
            
            return x

#----------------------------------------------------------------------------------------------------------------------------#
    def forward(self, x):
        
        mu, logvar        =    self.encode(x)
        z                 =    self.reparameterize(mu, logvar)
        reconstructed     =    self.decode(z)  
        
        kl_loss           =    -0.5 * torch.sum(1 + logvar - mu**2 - torch.exp(logvar), dim=-1)
        
        return reconstructed, kl_loss.mean()
    



In [ ]:
def weights_init(model):
        
        for name, module in model.named_modules():

            # Convolutional layers
            if isinstance(module, nn.Conv2d):
                init.kaiming_normal_(module.weight, mode='fan_in', nonlinearity='relu')  # Kaiming initialization for conv layers
                if module.bias is not None:
                    init.zeros_(module.bias)  # Initialize biases to zero

            # Transposed Convolutional layers (Deconvolution)
            elif isinstance(module, nn.ConvTranspose2d):
                init.kaiming_normal_(module.weight, mode='fan_in', nonlinearity='relu')  # Kaiming for transpose conv
                if module.bias is not None:
                    init.zeros_(module.bias)

            # Fully connected (Linear) layers
            elif isinstance(module, nn.Linear):
                init.xavier_normal_(module.weight)  # Xavier initialization for FC layers
                if module.bias is not None:
                    init.zeros_(module.bias)  # Initialize biases to zero

            # BatchNorm layers (if any)
            elif isinstance(module, nn.BatchNorm2d) or isinstance(module, nn.BatchNorm1d):
                init.ones_(module.weight)  # Initialize BatchNorm scale parameter to 1
                init.zeros_(module.bias)  # Initialize BatchNorm bias parameter to 0

            # Transformer layers (self-attention and feedforward)
            elif isinstance(module, nn.TransformerEncoderLayer):
                # Initialize self-attention layers
                init.xavier_normal_(module.self_attn.in_proj_weight)  # Attention projection weights
                init.xavier_normal_(module.self_attn.out_proj.weight)  # Attention output projection weights
                if module.self_attn.in_proj_bias is not None:
                    init.zeros_(module.self_attn.in_proj_bias)  # Initialize biases to zero

                # Initialize feedforward layers (MLP within the Transformer)
                init.kaiming_normal_(module.linear1.weight, mode='fan_in', nonlinearity='relu')  # Kaiming for the first FC layer
                init.kaiming_normal_(module.linear2.weight, mode='fan_in', nonlinearity='relu')  # Kaiming for the second FC layer
                if module.linear1.bias is not None:
                    init.zeros_(module.linear1.bias)  # Initialize biases to zero
                if module.linear2.bias is not None:
                    init.zeros_(module.linear2.bias)

            # Transformer Encoder
            elif isinstance(module, nn.TransformerEncoder):
                # Initialize all transformer encoder layers
                for encoder_layer in module.layers:
                    weights_init(encoder_layer)  # Recursively initialize each transformer encoder layer

            # For any other modules, initialize as needed
            else:
                pass  # Default behavior for modules that don't require special initialization

<div span style="background-color:yellow;    color:red;      text-align:center;    padding:5px;">
<h2>  To Employ GPU </h2>
</div>

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
device

<div span style="background-color:yellow;    color:red;      text-align:center;    padding:5px;">
<h2>  Move Model to GPU  </h2>
</div>

In [ ]:
model   =   Beta_VAE_ViT(ch=3, img_size=224, patch_size=16, embed_dim=64, latent_dim = 64, n_layers=6, dropout=0.1, heads=8).to(device)

In [ ]:
num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print("Number of trainable parameters in the model:", num_params)

In [ ]:
weights_init(model)

<div span style="background-color:yellow;    color:red;      text-align:center;    padding:5px;">
<h2>  To Save Weights and Losses : </h2>
</div>

In [ ]:
path_model   =  "/home/turgay/Turgay/Academic/2024-2025/Fall/Generative_Models/Final_Project/Final_Report/codes/weights/Beta_VAE_transformers_weights_1.pth"
path_losses  =  "/home/turgay/Turgay/Academic/2024-2025/Fall/Generative_Models/Final_Project/Final_Report/codes/weights/Beta_VAE_transformers_losses_1.pth"

In [ ]:
torch.save({
            'model_state_dict': model.state_dict(),
            'best_loss'       : 9999999999999,
        }, path_model)   

In [ ]:
torch.save({
            'train_loss_rec': [],
            'train_loss_kl' : [],
            'train_loss'    : [],
    
            'val_loss_rec'  : [],
            'val_loss_kl'   : [],
            'val_loss'      : [],
    
            'epochs'        : [],
    
        }, path_losses)    

In [ ]:
checkpoint_weights   =   torch.load(path_model, weights_only=True) 
checkpoint_losses    =   torch.load(path_losses, weights_only=True)  

model.load_state_dict(checkpoint_weights['model_state_dict'])

best_loss       =  checkpoint_weights['best_loss'] 

train_loss      =  checkpoint_losses['train_loss']
train_loss_kl   =  checkpoint_losses['train_loss_kl']
train_loss_rec  =  checkpoint_losses['train_loss_rec']

val_loss        =  checkpoint_losses['val_loss'] 
val_loss_kl     =  checkpoint_losses['val_loss_kl'] 
val_loss_rec    =  checkpoint_losses['val_loss_rec'] 

epochs          =  checkpoint_losses['epochs'] 

In [ ]:
best_loss

<div span style="background-color:yellow;    color:red;      text-align:center;    padding:5px;">
<h2>  Parameters :  </h2>
</div>

In [ ]:
#------------------------------------
beta_start    =   0.000001
beta_end      =   10

num_epochs    =   100
#------------------------------------
mu_values     =   [] 
logvar_values =   []

<div span style="background-color:yellow;    color:red;      text-align:center;    padding:5px;">
<h2>  Schedulers for Beta </h2>
</div>

In [ ]:
def linear_schedule(epoch):
    return beta_start + ((beta_end - beta_start) / num_epochs) * epoch 
#---------------------------------------------------------------------------------#
def exponential_schedule(epoch):
    return beta_start * ((beta_end / beta_start) ** (epoch / num_epochs))

<div span style="background-color:yellow;    color:red;      text-align:center;    padding:5px;">
<h2>   Train and Validation Function </h2>
</div>

In [ ]:
def train(num_epochs=10, beta=0, patience=10, counter=0, learning_rate=0.001, best_loss=99999999 ):
    
    
    optimizer =   torch.optim.Adam(model.parameters(), lr=learning_rate)
    criterion =   nn.MSELoss()
    scaler    =   GradScaler() 
    
    
    for epoch in range(num_epochs):

        beta   =   beta 

        model.train()

        total_loss_train      =   0.0
        total_loss_train_kl   =   0.0
        total_loss_train_rec  =   0.0

        for batch in tqdm.tqdm(train_loader):

            images   = batch

            images   =  images.to(device)


#--------------------------------------------------------------------------------------------------------------------------#

            with autocast(device_type='cuda'):

                img_recon, kl_loss   =   model(images)

                img_rec_loss         =   criterion(img_recon, images)

                loss = img_rec_loss + kl_loss * beta  

#--------------------------------------------------------------------------------------------------------------------------#
            scaler.scale(loss).backward()
            #torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)  
            scaler.step(optimizer)
            scaler.update()

            total_loss_train      +=   loss.item()
            total_loss_train_kl   +=   kl_loss.item()
            total_loss_train_rec  +=   img_rec_loss.item()

        total_loss_train     /= len(train_loader)
        total_loss_train_kl  /= len(train_loader)
        total_loss_train_rec /= len(train_loader)

        train_loss.append(total_loss_train)
        train_loss_kl.append(total_loss_train_kl)
        train_loss_rec.append(total_loss_train_rec)

        total_loss_train_tensor = torch.tensor(total_loss_train)

        if torch.isnan(total_loss_train_tensor):
            print("nan value is encountered !")

            break

        print( "-------------------------------------------------------------------------------")
        print(f"|  Epoch [{epoch+1}/{num_epochs}]          |        Total Train Loss : {total_loss_train:.4f}           |")
        print( "-------------------------------------------------------------------------------")

        print( "-------------------------------------------------------------------------------")
        print(f"|  Epoch [{epoch+1}/{num_epochs}]          |        Total KL Loss : {total_loss_train_kl:.4f}              |")
        print( "-------------------------------------------------------------------------------")

        print( "-------------------------------------------------------------------------------")
        print(f"|  Epoch [{epoch+1}/{num_epochs}]          |        Total REC Loss : {total_loss_train_rec:.4f}              |")
        print( "-------------------------------------------------------------------------------")

#--------------------------------------------------------------------------------------------------------------------------#
#--------------------------------------------------------------------------------------------------------------------------#
#--------------------------------------------------------------------------------------------------------------------------#

        model.eval()

        total_loss_val      =   0.0
        total_loss_val_kl   =   0.0
        total_loss_val_rec  =   0.0
        total_loss_val_reg  =   0.0

        with torch.no_grad():

            for batch in tqdm.tqdm(val_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):

                images   =  batch

                images   =  images.to(device)
#--------------------------------------------------------------------------------------------------------------------------#

                img_recon, kl_loss   =   model(images)

                img_rec_loss         =   criterion(img_recon, images)

                loss = img_rec_loss + kl_loss * beta  


                total_loss_val      +=   loss.item()
                total_loss_val_kl   +=   kl_loss.item()
                total_loss_val_rec  +=   img_rec_loss.item()


        total_loss_val     /= len(val_loader)
        total_loss_val_kl  /= len(val_loader)
        total_loss_val_rec /= len(val_loader)

        val_loss.append(total_loss_val)
        val_loss_kl.append(total_loss_val_kl)
        val_loss_rec.append(total_loss_val_rec)


        print( "-------------------------------------------------------------------------------")
        print(f"|  Epoch [{epoch+1}/{num_epochs}]         |       Total Validation Loss : {total_loss_val:.4f}           |")
        print( "-------------------------------------------------------------------------------")

        print( "-------------------------------------------------------------------------------")
        print(f"|  Epoch [{epoch+1}/{num_epochs}]         |       Total KL Loss : {total_loss_val_kl:.4f}              |")
        print( "-------------------------------------------------------------------------------")

        print( "-------------------------------------------------------------------------------")
        print(f"|  Epoch [{epoch+1}/{num_epochs}]         |       Total REC Loss : {total_loss_val_rec:.4f}             |")
        print( "-------------------------------------------------------------------------------")

#--------------------------------------------------------------------------------------------------------------------------#
#--------------------------------------------------------------------------------------------------------------------------#
#--------------------------------------------------------------------------------------------------------------------------#

        if len(val_loss) >= 2:

            res   =   (  (val_loss[-2] - val_loss[-1]) / val_loss[-2] ) * 100 
            print( "-------------------------------------------------------------------------------")
            print(f"|              Change in loss is      %   {res:.2f}                               |")
            print( "-------------------------------------------------------------------------------")

        if total_loss_val < best_loss:
            print("*************...saving best model *************")
            best_loss = total_loss_val 
            torch.save({
                'model_state_dict': model.state_dict(),
                'best_loss': best_loss,
            }, path_model)   

        epochs.append(epoch)
        
        torch.save({
                'train_loss_rec'   : train_loss_rec,
                'train_loss_kl'    : train_loss_kl,
                'train_loss'       : train_loss,

                'val_loss_rec'     : val_loss_rec,
                'val_loss_kl'      : val_loss_kl,
                'val_loss'         : val_loss,

                'epochs'           : epochs, 

            }, path_losses) 

#--------------------------------------------------------------------------------------------------------------------------#
        if (len(val_loss) >= 2) and (val_loss[-2] > val_loss[-1]):
            counter = 0
        else:
            counter += 1
            if counter >= patience:
                print("Early stopping!")
                break
            
#--------------------------------------------------------------------------------------------------------------------------#
        torch.cuda.empty_cache()


<div span style="background-color:yellow;    color:red;      text-align:center;    padding:5px;">
<h2>   Start Train and Validation </h2>
</div>

In [ ]:
train(num_epochs=100, beta=0, patience=10, counter=0, learning_rate=0.001)

<div span style="background-color:yellow;    color:red;      text-align:center;    padding:5px;">
<h2>  To Plot Train and Validation Losses </h2>
</div>

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(6,6))

ax.plot(train_loss, "b-", label="Train Loss")
ax.plot(val_loss, "r-", label="Validation Loss")
ax.set_xlabel("Epochs")
ax.set_ylabel("MSE_Loss")
ax.legend()

<div span style="background-color:yellow;    color:red;      text-align:center;    padding:5px;">
<h2>   To Test Reconstruction Quality </h2>
</div>

In [ ]:
with torch.no_grad():
    for img in val_loader:
        img    = img.to(device)

        recon, kl  = model(img)
        break

In [ ]:
img.shape, recon.shape

In [ ]:
img.max(), recon.max()

In [ ]:
import matplotlib.pyplot as plt

plt.figure(dpi=250)

fig, ax = plt.subplots(2, 7, figsize=(15, 4))

for i in range(7):
	ax[0, i].imshow(img[i].reshape(224,224,3).cpu().numpy())
	ax[1, i].imshow(recon[i].reshape(224, 224, 3).cpu().numpy())
	ax[0, i].axis('OFF')
	ax[1, i].axis('OFF')
plt.show()


<div span style="background-color:yellow;    color:red;      text-align:center;    padding:5px;">
<h2>   Test Latent Space </h2>
</div>

In [ ]:
e = model.eval()

<div span style="background-color:yellow;    color:red;      text-align:center;    padding:1px;">
<h2>   The following image will be sent to the latent space: </h2>
</div>

In [ ]:

plt.figure(dpi=250)

fig, ax = plt.subplots(1, 1, figsize=(4, 4))

ax.imshow(img[1].reshape(224,224,3).cpu().numpy())

plt.show()


<div span style="background-color:yellow;    color:red;      text-align:center;    padding:1px;">
<h2>   Create Vector z: </h2>
</div>

In [ ]:
mu, logvar = model.encode(img[1].reshape(1, 3, 224, 224))

In [ ]:
z = model.reparameterize(mu, logvar)

In [ ]:
z.shape

<div span style="background-color:yellow;    color:red;      text-align:center;    padding:1px;">
<h2>   Traversal Function: </h2>
</div>

In [ ]:
def latent_space_traversal(model, z, latent_dim, steps=10, range_val=3):

    device = next(model.parameters()).device  

    fig, axs = plt.subplots(latent_dim, steps, figsize=(steps * 2, latent_dim * 2))

    for dim in range(latent_dim):
        for step, val in enumerate(torch.linspace(-range_val, range_val, steps)):
 
            z_traversal             =   z.clone()
            z_traversal[0, dim]  =   val 

            with torch.no_grad():
                
                recon              = model.decode(z_traversal).squeeze(0).cpu().numpy()

            axs[dim, step].imshow(recon.reshape(224, 224, 3))
            axs[dim, step].axis("off") 

In [ ]:
latent_space_traversal(model, z, 16) 